In [319]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.lm.preprocessing import pad_both_ends
stopwords = nltk.corpus.stopwords.words('english')
from nltk.lm.api import Smoothing
import math
import string
import pytest
import string
import pandas as pd
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.lm import Vocabulary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [320]:
import re
def clean(listoftweets):
    listoffiltered=[]
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    for i in range(0,len(listoftweets)):
         
         
         listoftweets[i]=listoftweets[i].replace('@USER', '') #Remove mentions (@USER)
         
         listoftweets[i]=listoftweets[i].replace('URL', '')
         listoftweets[i]=listoftweets[i].replace('&amp', 'and')
         listoftweets[i]=listoftweets[i].replace('&lt', '')
         listoftweets[i]=listoftweets[i].replace('&gt', '')
         listoftweets[i]=listoftweets[i].replace('\d+', '')
         listoftweets[i] = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          (listoftweets[i])) 
         listoftweets[i]=emoji_pattern.sub(r'', listoftweets[i])
         listoftweets[i]=listoftweets[i].translate(str.maketrans('', '', string.punctuation))
         listoftweets[i]=listoftweets[i].strip()
         listoftweets[i] = re.sub(' +', ' ', listoftweets[i])
         listoftweets[i]=listoftweets[i].lower()
         word_tokens = word_tokenize(listoftweets[i])
         filteredsen = [w for w in word_tokens if w not in stopwords and w != ' ']
         listoffiltered.append(filteredsen)
    return listoffiltered
    

In [321]:
def train_LM(pathtotrainfile):
   fulldata=pd.read_csv(pathtotrainfile,sep='\t')
   columns = ['id','subtask_b','subtask_c']
   fulldata.drop(columns, axis=1, inplace=True)
   data_off = fulldata[fulldata['subtask_a'] == 'OFF']
   data_nff = fulldata[fulldata['subtask_a'] == 'NOT']
   listoftweets1 = []
   listoftweets2 = []
   listoftweets3 = []
   for i in range(0,len(fulldata)):
      listoftweets1.append(fulldata.iloc[i,0])
   for i in range(0,len(data_off)):
      listoftweets2.append(data_off.iloc[i,0])
   for i in range(0,len(data_nff)):
       listoftweets3.append(data_nff.iloc[i,0])

   list1 = clean(listoftweets1)
   list2 = clean(listoftweets2)
   list3 = clean(listoftweets3)
  #  cleanedio = remove_stopwords(list1)
  #  cleanedio_off = remove_stopwords(list2)
  #  cleanedio_nff = remove_stopwords(list3)
  # doing padding and generating bigrams on each data of LM 
   train, vocab = padded_everygram_pipeline(2, list1)
   train1, vocab1 = padded_everygram_pipeline(2, list2)
   train2, vocab2 = padded_everygram_pipeline(2, list3)
   # fitting the model
   Lmfull = MLE(2)
   Lmfull.fit(train,vocab)
   Lmoff = MLE(2)
   Lmoff.fit(train1,vocab1)
   Lmnff = MLE(2)
   Lmnff.fit(train2,vocab2)
   return Lmfull, Lmoff, Lmnff  

In [322]:
Lmfull, Lmoff, Lmnff = train_LM('olid-training-v1.0.tsv')

In [323]:
def Average(lst):
    return sum(lst) / len(lst)

In [324]:
def write_file(df, filename):
    df.to_csv(filename, sep=',',index=None)

In [325]:
from nltk.lm.preprocessing import pad_both_ends
from nltk.util import bigrams
import numpy as np
def test_LM(pathtotestfile, model):
   fulldata=pd.read_csv(pathtotestfile,sep='\t')
   columns = ['id']
   fulldata.drop(columns, axis=1, inplace=True)
   
   labeltest = pd.read_csv('/content/labels-levela.csv', header=None)
   
   labeltest1 = labeltest.iloc[:,1]
   fulldata['subtask_a']=labeltest1
   df1=fulldata
   
   data_off = df1[df1['subtask_a'] == 'OFF']
   data_nff = df1[df1['subtask_a'] == 'NOT']
   listoftweetstest1 = []
   listoftweetstest2 = []
   listoftweetstest3 = []
   for i in range(0,len(fulldata)):
      listoftweetstest1.append(fulldata.iloc[i,0])
   for i in range(0,len(data_off)):
      listoftweetstest2.append(data_off.iloc[i,0])
   for i in range(0,len(data_nff)):
       listoftweetstest3.append(data_nff.iloc[i,0])
   # preprocessing of tweets    
 
   listtest1 = clean(listoftweetstest1)
   listtest2 = clean(listoftweetstest2)
   listtest3 = clean(listoftweetstest3)
   cleanediotest = listtest1
   cleanedio_offtest = listtest2
   cleanedio_nfftest = listtest3
   bigramtest1=[]
   bigramtest2=[]
   bigramtest3=[]
   for each in cleanediotest:
     bigramtest1.append(list(bigrams(pad_both_ends(each, n=2))))
   for each in cleanedio_offtest:
     bigramtest2.append(list(bigrams(pad_both_ends(each, n=2))))
   for each in cleanedio_nfftest:
     bigramtest3.append(list(bigrams(pad_both_ends(each, n=2))))
   mletest1=[]
   mletest2=[]
   mletest3=[]
   # calculation of mle score and appendig to respective mletest list according to data category and ignoring zero probability using respective model
   for each in bigramtest1:
      gh=1
      for bn in each:
         jk12 = abs(model.score(bn[1],[bn[0]]))
         if jk12 !=0 :
           gh = gh * ((abs(jk12)))
      mletest1.append(gh)  
   for each1 in bigramtest2:
      gh1=1
      for bn1 in each1:
         jk = abs(model.score(bn1[1],[bn1[0]]))
         if jk != 0:
           gh1 = gh1 * (abs(jk)) 
      mletest2.append(gh1)   
   for each2 in bigramtest3:
      gh2=1
      for bn2 in each2:
          jk1 = abs(model.score(bn2[1],[bn2[0]]))
          if jk1 != 0:
           gh2 = gh2 * ((abs(jk1)))
      mletest3.append((gh2))   
   fulldata1 =  pd.DataFrame()
   data_off1 = pd.DataFrame()
   data_nff1 = pd.DataFrame()
   fulldata1['mlescoreonfull']=mletest1
   data_off1['mlescoreonoff']=mletest2
   data_nff1['mlescoreonnff']=mletest3
   fulldata1.to_csv('fullmodel.csv', sep='\t', index=False)
   data_off1.to_csv('offmodel.csv', sep='\t',index=False) 
   data_nff1.to_csv('notoffmodel.csv', sep='\t',index=False)
   print("Avgscoremodel","fulldata", abs(Average(mletest1)))
   print("Avgscoremodel","offdata", abs(Average(mletest2)))
   print("Avgscoremodel","nffdata", abs(Average(mletest3)))

In [326]:
test_LM('testset-levela.tsv',Lmfull)


Avgscoremodel fulldata 0.1170799470816836
Avgscoremodel offdata 0.08128102248200787
Avgscoremodel nffdata 0.13093759531381607


In [327]:
test_LM('testset-levela.tsv',Lmoff)

Avgscoremodel fulldata 0.2314758040772122
Avgscoremodel offdata 0.14837365708643893
Avgscoremodel nffdata 0.263644377105899


In [328]:
test_LM('testset-levela.tsv',Lmnff)

Avgscoremodel fulldata 0.14174253349455804
Avgscoremodel offdata 0.11943627460846734
Avgscoremodel nffdata 0.1503772143536898
